In [3]:
import torch
import torch.nn as nn

# Bi-GRU 모델 정의
class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=4):
        super(BiGRU, self).__init__()
        self.bigru = nn.GRU(input_size, hidden_size, num_layers, 
                           batch_first=True, bidirectional=True)
    
    def forward(self, x):
        output, hidden = self.bigru(x)
        return output, hidden

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=4):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, 
                          batch_first=True)
    
    def forward(self, x):
        output, hidden = self.gru(x)
        return output, hidden


# 차원 변화 확인
input_size = 100  # 입력 특성 차원
hidden_size = 256  # 은닉 상태 차원
seq_length = 20   # 시퀀스 길이
batch_size = 32   # 배치 크기

# 모델 생성
model0 = GRU(input_size, hidden_size)
model = BiGRU(input_size, hidden_size)

# 더미 입력 데이터 생성
input_data = torch.randn(batch_size, seq_length, input_size)

print(f"Batch_size: {batch_size}, \nSequence Length: {seq_length}")
print(f"Hidden: {hidden_size}")

# Bi-GRU를 통과시켜 representation 추출
with torch.no_grad():
    output0, hidden0 = model0(input_data)
    output, hidden = model(input_data)

fbhidden0 = hidden0[-1]  # GRU의 은닉 상태를 합칩니다.
fbhidden = torch.cat([hidden[-2], hidden[-1]], dim=-1)  # 양방향 GRU의 은닉 상태를 합칩니다.

print()
print("GRU----------------------")
print(f"입력 차원: {input_data.shape}")
print(f"출력 차원: {output0.shape}")
print(f"은닉 상태 차원: {hidden0.shape}")
print(f"마지막 은닉 상태 차원: {fbhidden0.shape}")

print()
print("Bi-GRU-------------------")
print(f"입력 차원: {input_data.shape}")
print(f"출력 차원: {output.shape}")
print(f"은닉 상태 차원: {hidden.shape}")
print(f"마지막 은닉 상태 차원: {fbhidden.shape}")

Batch_size: 32, 
Sequence Length: 20
Hidden: 256

GRU----------------------
입력 차원: torch.Size([32, 20, 100])
출력 차원: torch.Size([32, 20, 256])
은닉 상태 차원: torch.Size([4, 32, 256])
마지막 은닉 상태 차원: torch.Size([32, 256])

Bi-GRU-------------------
입력 차원: torch.Size([32, 20, 100])
출력 차원: torch.Size([32, 20, 512])
은닉 상태 차원: torch.Size([8, 32, 256])
마지막 은닉 상태 차원: torch.Size([32, 512])


In [4]:
class EventSeqEncoder(nn.Module):
    def __init__(self, attr_dims:list, hidden_dim:int, num_layers:int):
        super(EventSeqEncoder, self).__init__()

        self.attr_embs = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in attr_dims[1:]])
        self.gru = nn.GRU(input_size=hidden_dim*len(attr_dims[1:]),
                          hidden_size=hidden_dim,
                          num_layers=num_layers, 
                          dropout=0.3, batch_first=True, bidirectional=True)
        
    
    def forward(self, seq):

        print("Before Shape:", seq.shape)

        # seq: (Batch_size, Seq_len, Attr_num)

        embedded = []
        for i, m in enumerate(self.attr_embs):
            embedded.append(m(seq[:, :, i]))

        seq = torch.cat(embedded, dim=2)  # (Batch_size, Seq_len, hidden_dim * (Attr_num-1))

        print("After Embedding:", seq.shape)

        self.gru.flatten_parameters()
        output, hidden = self.gru(seq)

        fwd, bwd = hidden[-2, :, :], hidden[-1, :, :]

        print("After GRU Output:", output.shape)
        print("After GRU Hidden:", hidden.shape)
        print("After GRU Fwd:", fwd.shape)
        print("After GRU Bwd:", bwd.shape)

# Sample Input
attr_dims = [10, 20, 30]  # 예시로 각 속성의 차원
hidden_dim = 64  # 은닉 상태 차원
num_layers = 2  # GRU 레이어 수
seq_len = 5  # 시퀀스 길이
batch_size = 3  # 배치 크기

seq = torch.randint(0, 10, (batch_size, seq_len, len(attr_dims)))  # 예시 시퀀스 데이터
encoder = EventSeqEncoder(attr_dims, hidden_dim, num_layers)
encoder(seq)


Before Shape: torch.Size([3, 5, 3])
After Embedding: torch.Size([3, 5, 128])
After GRU Output: torch.Size([3, 5, 128])
After GRU Hidden: torch.Size([4, 3, 64])
After GRU Fwd: torch.Size([3, 64])
After GRU Bwd: torch.Size([3, 64])


In [5]:
import torch

x = torch.randn(10, 20)  # 예시로 랜덤 텐서 생성

import torch.nn as nn

sm = nn.Softmax(dim=1)

res = sm(x)

print(x.shape)
print(res.shape)

torch.Size([10, 20])
torch.Size([10, 20])


In [3]:
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool

class GraphEncoder(nn.Module):
    def __init__(self, attr_dims:list, hidden_dim:int):
        super(GraphEncoder, self).__init__()

        self.acts_embedding = nn.Embedding(attr_dims[0], hidden_dim)
        self.conv1 = GATConv(in_channels=hidden_dim,
                            out_channels=hidden_dim,
                            heads=2, dropout=0.3)
        self.act1 = nn.ELU()
        self.do1 = nn.Dropout(p=0.3)    

        self.conv2 = GATConv(in_channels=hidden_dim * 2,
                            out_channels=hidden_dim,
                            heads=2, dropout=0.3)
        self.act2 = nn.ELU()
        self.do2 = nn.Dropout(p=0.3)
        self.projection = nn.Linear(hidden_dim * 2, hidden_dim)     # Shape(A, H)

    def forward(self, x, edge_index, batch_idx):
        x = self.acts_embedding(x)
        h = self.conv1(x, edge_index)
        h = self.act1(h)
        h = self.do1(h)

        h = self.conv2(h, edge_index)
        h = self.act2(h)
        h = self.do2(h)

        h = self.projection(h)              # Shape(A, H)
        z = global_mean_pool(h, batch_idx)  # Shape(B, H)
        return h, z


class EventSeqEncoder(nn.Module):
    def __init__(self, attr_dims:list, hidden_dim:int, num_layers:int):
        super(EventSeqEncoder, self).__init__()

        self.attr_embs = nn.ModuleList([nn.Embedding(dim, hidden_dim) for dim in attr_dims[1:]])
        self.gru = nn.GRU(input_size=hidden_dim*len(attr_dims[1:]),
                          hidden_size=hidden_dim,
                          num_layers=num_layers, 
                          dropout=0.3, batch_first=True, bidirectional=True)


    def forward(self, seq):
        self.gru.flatten_parameters()
        # seq: (Batch_size, Seq_len, Attr_num)
        embs = []
        for i, m in enumerate(self.attr_embs):
            embs.append(m(seq[:, :, i]))
        
        seq_embs = torch.cat(embs, dim=2)
        # seq_embs: (batch_size, seq_len, num_attrs * hidden_dim)

        out, hidden = self.gru(seq_embs)

        fwd, bwd = hidden[-2, :, :], hidden[-1, :, :]   # Shape(batch_size, hidden)
        return out, fwd, bwd


In [5]:
# 예시 입력 텐서 생성 및 모델 입출력 테스트

# GraphEncoder 테스트용 입력
# 노드 feature: action id (정수), edge_index: (2, num_edges), batch_idx: (num_nodes,)
num_nodes = 6
num_edges = 8
action_ids = torch.randint(0, attr_dims[0], (num_nodes,))  # (num_nodes,)
edge_index = torch.randint(0, num_nodes, (2, num_edges))    # (2, num_edges)
batch_idx = torch.randint(0, 2, (num_nodes,))               # (num_nodes,)

graph_encoder = GraphEncoder(attr_dims, hidden_dim)
h, z = graph_encoder(action_ids, edge_index, batch_idx)
print("GraphEncoder output shapes:")
print("h (node embeddings):", h.shape)  # (num_nodes, hidden_dim)
print("z (graph embeddings):", z.shape) # (num_graphs, hidden_dim)

# EventSeqEncoder 테스트용 입력
# 시퀀스: (batch_size, seq_len, attr_num)
batch_size = 4
seq_len = 5
attr_num = len(attr_dims)
seq = torch.randint(0, 10, (batch_size, seq_len, attr_num))  # (batch_size, seq_len, attr_num)

event_seq_encoder = EventSeqEncoder(attr_dims, hidden_dim, num_layers)
out, fwd, bwd = event_seq_encoder(seq)
print("\nEventSeqEncoder output shapes:")
print("out (all hidden states):", out.shape)   # (batch_size, seq_len, hidden_dim*2)
print("fwd (forward last hidden):", fwd.shape) # (batch_size, hidden_dim)
print("bwd (backward last hidden):", bwd.shape)# (batch_size, hidden_dim)

GraphEncoder output shapes:
h (node embeddings): torch.Size([6, 16])
z (graph embeddings): torch.Size([2, 16])

EventSeqEncoder output shapes:
out (all hidden states): torch.Size([4, 5, 32])
fwd (forward last hidden): torch.Size([4, 16])
bwd (backward last hidden): torch.Size([4, 16])
